- grid search
 
but it is difficult in multidimensional...


then you can do a random walk search to explore the space of hyperparameters


then....

cross validation (k fold): compute the STANDARD DEVIATION associated to the accuracy, across the folds. then you can make a meaningful comparison.



AUGMENTATION:

pretend you dont have the function to generate the data. you need to create artificial ones.

small displacement and give it the same class as the original one. 
if done with moderation, it increases accuracy of the nn. if you do it too much, you risk. 

an example of augmentation is the rotation of an image with different angles (always the same image e.g. a tumor, or a cat... or whatever) but for the nn is a new sample 

#### Imports

In [10]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras_tuner 
from keras.optimizers import Adam, RMSprop, Nadam

#import matplotlib as mpl
import matplotlib.pyplot as plt
# default font
plt.rcParams['font.size'] = 13

%run useful.py

# training data
perc_train = 0.8

In [11]:
# Keras works with numpy arrays: just use them from the start

TYPE=3
# data point size
L=8 
# span of each component
B=10
x = np.loadtxt(filename("data",L,TYPE), delimiter=' ')
y = np.loadtxt(filename("labels",L,TYPE), delimiter=' ')
y = y.astype("int")
N = len(x)
# average and std.dev.
x_mean = np.mean(x,axis=0)
x_std  = np.std(x,axis=0)
# dim. of a sample
L = len(x[0])
N_train = int(perc_train * N)
print(f'data: {N}\ntrain: {N_train}')
#plot_data(x,y)
#print("before rescaling:\nmean value=", x_mean,"\nstd. dev.=",x_std)
def Standardize(x,m,s):
    """
    rescale each component using its mean and standard deviation
    """
    N = len(x)
    # assuming len(m)=len(s)=len(x[0])
    mm,ss = np.tile(m,(N,1)), np.tile(s,(N,1))
    return (x-mm)/ss
x = Standardize(x,x_mean,x_std)
print("after rescaling:\nmean value=", x.mean(axis=0),"\nstd. dev.=",x.std(axis=0))

data: 2000
train: 1600
after rescaling:
mean value= [ 2.53633226e-15  9.27924404e-16 -1.44884105e-15 -2.38371996e-15
 -2.85274582e-15 -2.02196593e-15 -1.09010023e-15 -1.53632662e-15] 
std. dev.= [1. 1. 1. 1. 1. 1. 1. 1.]


# Split training - test

In [12]:
(x_train, y_train) = (x[0:N_train],y[0:N_train])
(x_valid, y_valid) = (x[N_train:],y[N_train:])
print("Train:",len(x_train),"\t Validation:",len(x_valid))

Train: 1600 	 Validation: 400


### Define the model

In [ ]:
def build_model(hp):
    #chosen_activation = "relu"
    #chosen_dropout = 0.1
    
    chosen_activation = hp.Choice("activation", ["relu", "tanh", "sigmoid", "elu"])
    chosen_dropout = hp.Float("dropout_rate", min_value=0, max_value=0.2, step = 0.1)
    chosen_lr = hp.Float("learning_rate", min_value=1e-6, max_value=1e-1, sampling="log") 
     
    optimizers = {
        "adam": Adam(learning_rate=chosen_lr),
        "RMSprop": RMSprop(learning_rate=chosen_lr),
        "nadam": Nadam(learning_rate=chosen_lr),
    } 
    
    chosen_optimizer = hp.Choice("optimizer", ["adam", "RMSprop", "nadam"])        #other optimizers to try?


    model = Sequential()
    
    # input layer
    model.add( Dense(L, input_shape = (L, ), 
            # Tune the activation function to use.
            activation= chosen_activation,
        )
    )
    # first hidden
    model.add( Dense(20,
            activation=chosen_activation,
        )
    )
    model.add(Dropout(rate=chosen_dropout))
    # second hidden

    model.add( Dense(20,
            activation=chosen_activation,
        )
    )
    model.add(Dropout(rate=chosen_dropout))

    # third hidden
    model.add( Dense(20,
            activation=chosen_activation,
        )
    )
    model.add(Dropout(rate=chosen_dropout))

    # output layer
    model.add( Dense(1,
            activation="sigmoid",
        )
    )

    model.compile(
        optimizer=chosen_optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(keras_tuner.HyperParameters())

c:\Users\angel\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<Sequential name=sequential_1, built=True>

In [14]:
hp = keras_tuner.HyperParameters()


tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu'], 'ordered': False}
dropout_rate (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.2, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-06, 'conditions': [], 'min_value': 1e-06, 'max_value': 0.1, 'step': None, 'sampling': 'log'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop', 'nadam'], 'ordered': False}


In [15]:
tuner.search(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid))

Trial 5 Complete [00h 00m 51s]
val_accuracy: 0.8337499797344208

Best val_accuracy So Far: 0.9350000023841858
Total elapsed time: 00h 04m 33s


In [16]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

c:\Users\angel\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           180 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,113 (4.35 KB)

 Trainable params: 1,113 (4.35 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
tuner.results_summary()

Results summary
Results in my_dir\helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
activation: elu
dropout_rate: 0.0
learning_rate: 4.790651788961244e-05
optimizer: RMSprop
Score: 0.9350000023841858

Trial 4 summary
Hyperparameters:
activation: elu
dropout_rate: 0.1
learning_rate: 0.045098530718521754
optimizer: RMSprop
Score: 0.8337499797344208

Trial 3 summary
Hyperparameters:
activation: elu
dropout_rate: 0.2
learning_rate: 0.00032775211010339754
optimizer: nadam
Score: 0.8087499737739563

Trial 2 summary
Hyperparameters:
activation: tanh
dropout_rate: 0.2
learning_rate: 3.8856582747866586e-05
optimizer: adam
Score: 0.7212499976158142

Trial 0 summary
Hyperparameters:
activation: sigmoid
dropout_rate: 0.2
learning_rate: 0.0004320511258351558
optimizer: adam
Score: 0.5699999928474426


In [18]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
#model = build_model(best_hps[0])
# Fit with the entire dataset.
#x_all = np.concatenate((x_train, x_val))
#y_all = np.concatenate((y_train, y_val))
#model.fit(x=x_all, y=y_all, epochs=1)